In [1]:
import pandas as pd

# Uncomment to see full cell text
pd.set_option('display.max_colwidth', 0)

# Process Hand-cleaned Text Data

In [2]:
hand_cleaned = pd.read_csv('../data/interim/faq-hand-cleaned.csv')

## Separate Headings, Questions, Answers

In [3]:
def pairs(row):
    topic_qna = pd.DataFrame()
    heading = True
    h = q = a = ''
    qna = []
    
    for item in row.FAQ:
        if heading:
            if '?' not in item:
                h += ' ' + item
            else:
                heading = False
                q = item.strip()
                a = ''
        elif '?' in item:
            qna.append([q.strip(), a.strip()])
            q = item
            a = ''
        else:
            a += ' ' + item
            
    if q:
        qna.append([q.strip(), a.strip()])

    if h:
        qna = [[row.Topic+'?', h.strip()]] + qna
    row['heading'] = h.strip()
    row['qna'] = qna
    
    return row

faq = hand_cleaned.copy()
faq.FAQ = faq.FAQ.apply(lambda x: x.split('\n\n'))                
faq = faq.apply(pairs, axis=1)
faq

Topic  \
0    Affordable Homeownership                                           
1    Affordable Rental Opportunities                                    
2    Aircraft Noise                                                     
3    Airport                                                            
4    Alley Maintenance                                                  
5    Americans for the Arts Convention Scholarships                     
6    Americans with Disabilities Act (ADA)                              
7    Americans with Disabilities Act (ADA) Trails                       
8    Animal Noise (Barking Dogs)                                        
9    Animal Protection, Rules and Regulations                           
10   Animal Shelter                                                     
11   Ask-a-Librarian                                                    
12   Auction Licensing Instructions                                     
13   Backflow Prevention - Drinking Water                               
14   Bear Resistant Container Required                                  
15   Bears                                                              
16   Bicycling                                                          
17   Birth and Death Certificate Request                                
18   Block Party                                                        
19   Boulder Arts Commission Grants                                     
20   Boulder City Limits                                                
21   Boulder Civic Area                                                 
22   Boulder Convention and Visitors Bureau                             
23   Boulder Measures                                                   
24   Boulder Office of Emergency Management                             
25   Boulder Planning                                                   
26   Boulder Reservoir                                                  
27   Boulder Revised Code                                               
28   BoulderReads!                                                      
29   Broken Parking Meter or Pay Station                                
..                                   ...                                
220  Theater Rental and Marketing Assistance Grants                     
221  Towed/Impounded Vehicles                                           
222  Traffic Signals                                                    
223  Traffic Signs and Markings                                         
224  Train Noise and Quiet Zones                                        
225  Transit (Bus Service)                                              
226  Transportation Flood Recovery                                      
227  Transportation Projects                                            
228  Trash Concerns                                                     
229  Tree Branch Pick-Up                                                
230  Twenty Ninth Street                                                
231  Visitors                                                           
232  Volunteer Opportunities                                            
233  Volunteering at the Library                                        
234  Wage Issues                                                        
235  Waste Hauling Services                                             
236  Water Being Pumped or Diverted Off Private Property to Elsewhere   
237  Water Seepage                                                      
238  Water System Maintenance                                           
239  Water Utility Billing                                              
240  Water, Wastewater, Stormwater/Drainageway Flood Recovery           
241  Website Issues                                                     
242  Weed Control and Tall Vegetation                                   
243  Wildlife Issue or Sighting                  

## Re-Stack QnA Pairs

In [4]:
stack = faq.apply(lambda x: pd.Series(x['qna']), axis=1).stack().reset_index(level=1, drop=True)
stack.name = 'question'
faq = faq.drop(['FAQ', 'qna', 'heading', 'Updated'], axis=1).join(stack).reset_index(drop=True)
faq['answer'] = faq.question.apply(lambda x: x[1])
faq['question'] = faq.question.apply(lambda x: x[0])
faq

Topic  \
0    Affordable Homeownership                                           
1    Affordable Homeownership                                           
2    Affordable Homeownership                                           
3    Affordable Homeownership                                           
4    Affordable Homeownership                                           
5    Affordable Homeownership                                           
6    Affordable Homeownership                                           
7    Affordable Homeownership                                           
8    Affordable Homeownership                                           
9    Affordable Homeownership                                           
10   Affordable Homeownership                                           
11   Affordable Rental Opportunities                                    
12   Affordable Rental Opportunities                                    
13   Aircraft Noise                                                     
14   Airport                                                            
15   Airport                                                            
16   Airport                                                            
17   Airport                                                            
18   Airport                                                            
19   Airport                                                            
20   Airport                                                            
21   Airport                                                            
22   Airport                                                            
23   Alley Maintenance                                                  
24   Americans for the Arts Convention Scholarships                     
25   Americans with Disabilities Act (ADA)                              
26   Americans with Disabilities Act (ADA) Trails                       
27   Animal Noise (Barking Dogs)                                        
28   Animal Noise (Barking Dogs)                                        
29   Animal Noise (Barking Dogs)                                        
..                           ...                                        
658  Trash Concerns                                                     
659  Trash Concerns                                                     
660  Tree Branch Pick-Up                                                
661  Twenty Ninth Street                                                
662  Visitors                                                           
663  Volunteer Opportunities                                            
664  Volunteering at the Library                                        
665  Wage Issues                                                        
666  Waste Hauling Services                                             
667  Water Being Pumped or Diverted Off Private Property to Elsewhere   
668  Water Seepage                                                      
669  Water System Maintenance                                           
670  Water Utility Billing                                              
671  Water, Wastewater, Stormwater/Drainageway Flood Recovery           
672  Website Issues                                                     
673  Weed Control and Tall Vegetation                                   
674  Weed Control and Tall Vegetation                                   
675  Weed Control and Tall Vegetation                                   
676  Weed Control and Tall Vegetation                                   
677  Wildlife Issue or Sighting                                         
678  Wood Burning Restrictions                                          
679  Wood Burning Restrictions                                          
680  Youth Opportunities                                                
681  Youth Opportunities Grants                  

In [5]:
faq.to_csv('../data/interim/faq-text-separated.csv', index=False)

# Write Excel files for QnA Maker, IBM Watson, etc.

In [6]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('../data/processed/faq-two-columns.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
faq[['question', 'answer']].to_excel(writer, sheet_name='faq-text-separated', index=False)

# Close the Pandas Excel writer and output the Excel file.
writer.save()

# Write FAQ in 2 Columns, Q and A

In [7]:
faq[['question', 'answer']].to_csv('../data/interim/faq-two-columns.csv', index=False)